In [1]:
from nltk.corpus import cmudict, stopwords, words
from nltk import pos_tag
import random
import csv

In [2]:
cmudict.dict()['hello']

[['HH', 'AH0', 'L', 'OW1'], ['HH', 'EH0', 'L', 'OW1']]

In [20]:
def MED(w1, w2):
    
    cost = []
#     del_cost = 1
#     add_cost = 1
#     sub_cost = 2
    
    vowels = ['A', 'E', 'I', 'O', 'U']
    voiced = ['B', 'D', 'G', 'J', 'L', 'M', 'N', 'R', 'V', 'W', 'Y', 'Z']
    unvoiced = ['C', 'F', 'H', 'K', 'P', 'S', 'T']
    
    for i in range(len(w1)+1):
        x = []
        for j in range(len(w2)+1):
            x.append(0)
        cost.append(x)
    
    for i in range(len(w1)+1):
        cost[i][0] = i
    for j in range(len(w2)+1):
        cost[0][j] = j
        
    for i in range(1, len(w1)+1):
        for j in range(1, len(w2)+1):
            
            # if they are the same:
            if w1[i-1] == w2[j-1]:
                del_total = cost[i-1][j] + 10
                add_total = cost[i][j-1] + 10
                sub_total = cost[i-1][j-1]
            
            # if they are both vowels:
            elif w1[i-1][0] in vowels and w2[j-1][0] in vowels:
                del_total = cost[i-1][j] + 1
                add_total = cost[i][j-1] + 1
                sub_total = cost[i-1][j-1] + 2
                    
            # if only one is a vowel:
            elif w1[i-1][0] in vowels or w2[j-1][0] in vowels:
                del_total = cost[i-1][j] + 2
                add_total = cost[i][j-1] + 2
                sub_total = cost[i-1][j-1] + 3
                    
            # if both are voiced consonants:
            elif w1[i-1][0] in voiced and w2[j-1][0] in voiced:
                del_total = cost[i-1][j] + 1
                add_total = cost[i][j-1] + 1
                sub_total = cost[i-1][j-1] + 1
            
            # if both are unvoiced consonants:
            elif w1[i-1][0] in voiced and w2[j-1][0] in unvoiced:
                del_total = cost[i-1][j] + 1
                add_total = cost[i][j-1] + 1
                sub_total = cost[i-1][j-1] + 1
            
            # if one is voiced and the other is unvoiced:
            else:
                del_total = cost[i-1][j] + 1
                add_total = cost[i][j-1] + 1
                sub_total = cost[i-1][j-1] + 1
                    
            options = [del_total, add_total, sub_total]
            options.sort()
            cost[i][j] = options[0]
           
    return cost[-1][-1]

In [21]:
w = 'test'
w2 = 'beast'
res = MED(cmudict.dict()[w][0], cmudict.dict()[w2][0])

In [22]:
res

3

In [6]:
real_list = []
real_dict = []
with open('real_words_lower.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        real_list.append((row[0], eval(row[1])))
        real_dict.append(row[0])

In [7]:
real_dict = set(real_dict)

In [23]:
w = 'find'
word1 = cmudict.dict()[w][0]
options = []
for entry in real_list:
    price = MED(word1, entry[1])
    if price < 3 and w not in entry[0]:
        options.append(entry)

In [24]:
len(options)

70

In [25]:
options[20:40]

[('fire', ['F', 'AY1', 'R']),
 ('five', ['F', 'AY1', 'V']),
 ('fives', ['F', 'AY1', 'V', 'Z']),
 ('found', ['F', 'AW1', 'N', 'D']),
 ('fund', ['F', 'AH1', 'N', 'D']),
 ('fyke', ['F', 'AY1', 'K']),
 ('grind', ['G', 'R', 'AY1', 'N', 'D']),
 ('gyn', ['G', 'AY1', 'N']),
 ('hein', ['HH', 'AY1', 'N']),
 ('heinz', ['HH', 'AY1', 'N', 'Z']),
 ('hind', ['HH', 'AY1', 'N', 'D']),
 ('jain', ['JH', 'AY1', 'N']),
 ('kind', ['K', 'AY1', 'N', 'D']),
 ('line', ['L', 'AY1', 'N']),
 ('lined', ['L', 'AY1', 'N', 'D']),
 ('lived', ['L', 'AY1', 'V', 'D']),
 ('mild', ['M', 'AY1', 'L', 'D']),
 ('mind', ['M', 'AY1', 'N', 'D']),
 ('mine', ['M', 'AY1', 'N']),
 ('mines', ['M', 'AY1', 'N', 'Z'])]

In [29]:
def main():
    phrase = input('Enter a phrase: ').lower()
    word_list = phrase.split()
    
    choice_list = []
    for word in word_list:
        if word in stopwords.words('english'):
            choice_list.append([])
        elif word in real_dict:
            choices = []
            pos1 = pos_tag([word])[0][1]
            w1 = cmudict.dict()[word][0]
            for entry in real_list:
                pos2 = pos_tag([entry[0]])[0][1]
                if pos1 == pos2:
                    price = MED(w1, entry[1])
                    if price < 3 and word not in entry[0]:
                        choices.append(entry[0])
            choice_list.append(choices)
        else:
            choice_list.append([])
    
    # check to see if any subs were found:
    existing_choice = False
    for i in choice_list:
        if i:
            existing_choice = True
            break
    
    if existing_choice:
        position_choice = random.randrange(len(choice_list))
        while not choice_list[position_choice]:
            position_choice = random.randrange(len(choice_list))
        sub_choice = random.randrange(len(choice_list[position_choice]))
        sub = choice_list[position_choice][sub_choice]
        word_list[position_choice] = sub
        print(word_list)
    else:
        print('No suitable substitution found')

In [37]:
main()

Enter a phrase: code edits are required
['gnome', 'edits', 'are', 'required']
